# 🏡 İstanbul Emlak Yatırım Danışmanı
## AI SPARK HACKATHON - Model Eğitim Notebook

Bu notebook, ev fiyat tahmin modelinin eğitim sürecini adım adım göstermektedir.

## 1. Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb
import joblib
import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler yüklendi ✅")

## 2. Veri Yükleme ve İnceleme

In [ ]:
# Veriyi yükle
df = pd.read_csv('../hackathon_train_set.csv', sep=';')

print(f"Toplam kayıt: {len(df)}")
print(f"Toplam özellik: {len(df.columns)}")
df.head()

In [ ]:
# Hedef değişken inceleme
print("Price sütunu örnekleri:")
df['Price'].head(10)

## 3. Veri Temizleme (Data Cleaning)

In [ ]:
def clean_price(price_str):
    """Fiyat stringini sayısala çevirir."""
    if pd.isna(price_str):
        return np.nan
    clean_str = str(price_str).replace(' TL', '').replace('.', '').strip()
    try:
        return int(clean_str)
    except ValueError:
        return np.nan

df['Price'] = df['Price'].apply(clean_price)
df = df.dropna(subset=['Price'])

print(f"Temizleme sonrası kayıt: {len(df)}")
print(f"\nFiyat istatistikleri:")
df['Price'].describe()

In [ ]:
def clean_rooms(room_str):
    """'3+1' formatını toplam odaya çevirir."""
    if pd.isna(room_str):
        return np.nan
    try:
        parts = str(room_str).split('+')
        return sum(int(p) for p in parts)
    except:
        if "Stüdyo" in str(room_str):
            return 1
        return np.nan

def clean_age(age_str):
    """Bina yaşı aralıklarını sayısala çevirir."""
    if pd.isna(age_str):
        return np.nan
    s = str(age_str)
    if "0" == s: return 0
    if "1-5" in s: return 3
    if "5-10" in s: return 7.5
    if "11-15" in s: return 13
    if "16-20" in s: return 18
    if "21-25" in s: return 23
    if "26-30" in s: return 28
    if "31 ve üzeri" in s: return 35
    try:
        return int(s)
    except:
        return np.nan

def clean_balcony(val):
    if pd.isna(val): return 0
    if 'Available' in str(val) or 'Yes' in str(val): return 1
    return 0

## 4. Feature Engineering

In [ ]:
# m2 temizleme
for col in ['m² (Gross)', 'm² (Net)']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Oda sayısı
df['Rooms_Num'] = df['Number of rooms'].apply(clean_rooms)

# Bina yaşı
df['Age_Num'] = df['Building Age'].apply(clean_age)

# Oda başına m2
df['Room_Size_Ratio'] = df['m² (Net)'] / df['Rooms_Num'].replace(0, 1)

# Balkon
df['Balcony_Bool'] = df['Balcony'].apply(clean_balcony)

# Sayısal dönüşümler
for col in ['Number of bathrooms', 'Number of floors']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("Feature engineering tamamlandı ✅")

In [ ]:
# Kullanılacak özellikleri seç
keep_cols = [
    'Price', 'District', 'Neighborhood', 
    'm² (Net)', 'Rooms_Num', 'Age_Num', 'Room_Size_Ratio',
    'Floor location', 'Heating', 'Number of bathrooms', 'Number of floors',
    'Balcony_Bool', 'Elevator', 'Parking Lot', 'Security'
]

df_model = df[keep_cols].copy()

# Eksik değerleri doldur
binary_cols = ['Balcony_Bool', 'Elevator', 'Parking Lot', 'Security']
for col in binary_cols:
    df_model[col] = pd.to_numeric(df_model[col], errors='coerce').fillna(0)

num_cols = ['m² (Net)', 'Rooms_Num', 'Age_Num', 'Room_Size_Ratio', 'Number of bathrooms', 'Number of floors']
for col in num_cols:
    df_model[col] = df_model[col].fillna(df_model[col].median())

cat_cols = ['District', 'Neighborhood', 'Floor location', 'Heating']
df_model[cat_cols] = df_model[cat_cols].fillna("Unknown")

print(f"Final veri boyutu: {df_model.shape}")
df_model.info()

## 5. Outlier Temizleme

In [ ]:
# %1 ve %99 dilimler arası verileri tut
lower_bound = df_model['Price'].quantile(0.01)
upper_bound = df_model['Price'].quantile(0.99)

print(f"Alt sınır: {lower_bound:,.0f} TL")
print(f"Üst sınır: {upper_bound:,.0f} TL")

df_clean = df_model[(df_model['Price'] >= lower_bound) & (df_model['Price'] <= upper_bound)]
print(f"\nOutlier temizliği sonrası: {len(df_clean)} kayıt ({len(df_model) - len(df_clean)} silindi)")

## 6. Target Encoding

In [ ]:
class TargetEncoder:
    """Kategorik değişkenleri hedef değişkenin ortalaması ile encode eder."""
    
    def __init__(self, cols):
        self.cols = cols
        self.mapping = {}
    
    def fit(self, X, y):
        df = X.copy()
        df['_target'] = y.values
        
        global_mean = y.mean()
        
        for col in self.cols:
            means = df.groupby(col)['_target'].mean()
            self.mapping[col] = means.to_dict()
            self.mapping[col]['__global__'] = global_mean
        
        return self
    
    def transform(self, X):
        df = X.copy()
        for col in self.cols:
            global_mean = self.mapping[col]['__global__']
            df[col] = df[col].map(self.mapping[col]).fillna(global_mean)
        return df

print("Target Encoder sınıfı tanımlandı ✅")

## 7. Model Eğitimi

In [ ]:
# Veriyi ayır
X = df_clean.drop(columns=['Price'])
y = df_clean['Price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim seti: {len(X_train)} kayıt")
print(f"Validasyon seti: {len(X_val)} kayıt")

In [ ]:
# Target Encoding uygula
encoder = TargetEncoder(cols=['District', 'Neighborhood'])
encoder.fit(X_train, y_train)

X_train_enc = encoder.transform(X_train)
X_val_enc = encoder.transform(X_val)

print("Target Encoding uygulandı ✅")

In [ ]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['Floor location', 'Heating'])
    ],
    remainder='passthrough'
)

In [ ]:
# LightGBM modeli (Tuned parametreler)
lgb_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(
        n_estimators=700,
        max_depth=-1,
        learning_rate=0.05,
        num_leaves=50,
        subsample=0.9,
        colsample_bytree=0.7,
        min_child_samples=10,
        reg_alpha=0.1,
        reg_lambda=2,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ))
])

print("LightGBM modeli eğitiliyor...")
lgb_model.fit(X_train_enc, y_train)
print("Eğitim tamamlandı ✅")

## 8. Model Değerlendirme

In [ ]:
# Tahmin
y_pred = lgb_model.predict(X_val_enc)

# Metrikler
r2 = r2_score(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100

print("="*50)
print("📊 MODEL PERFORMANSI (Validasyon Seti)")
print("="*50)
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:,.0f} TL")
print(f"MAPE: {mape:.2f}%")

## 9. Model Kaydetme

In [ ]:
# Model ve encoder'ı kaydet
joblib.dump(lgb_model, '../models/model.pkl')
joblib.dump(encoder, '../models/target_encoder.pkl')

# İşlenmiş veriyi kaydet (arayüz için)
df_clean.to_pickle('../data/processed_data.pkl')

print("Model ve veriler kaydedildi ✅")
print("  → models/model.pkl")
print("  → models/target_encoder.pkl")
print("  → data/processed_data.pkl")

## 10. Örnek Tahmin

In [ ]:
# Örnek bir ev için tahmin
sample_house = pd.DataFrame({
    'District': ['Kadıköy'],
    'Neighborhood': ['Caferağa Mh.'],
    'm² (Net)': [120],
    'Rooms_Num': [4],
    'Age_Num': [10],
    'Floor location': ['High Rise'],
    'Heating': ['Natural Gas (Combi)'],
    'Number of bathrooms': [2],
    'Number of floors': [8],
    'Balcony_Bool': [1],
    'Elevator': [1],
    'Parking Lot': [1],
    'Security': [1],
    'Room_Size_Ratio': [30]
})

sample_enc = encoder.transform(sample_house)
predicted_price = lgb_model.predict(sample_enc)[0]

print("🏠 Örnek Ev Tahmini")
print("="*40)
print(f"Konum: Kadıköy, Caferağa")
print(f"Alan: 120 m², 4 Oda")
print(f"Özellikler: Asansör, Otopark, Güvenlik")
print(f"\n💰 Tahmin Edilen Değer: {predicted_price:,.0f} TL")

---
## 🎯 Sonuç

Bu notebook'ta:
1. Veri temizleme ve feature engineering yapıldı
2. Target Encoding ile kategorik değişkenler dönüştürüldü
3. LightGBM modeli hyperparameter tuning ile eğitildi
4. Model **R² = 0.8115** başarı oranı ile kaydedildi

Model artık Streamlit arayüzü ile kullanılmaya hazır!